# **Импорт библиотек**

In [2]:
from datetime import datetime, timedelta, time
import random
from babel.dates import format_date
import sys
import os

# **Логика "простого" алгоритма**

In [11]:
class BusDriver:
    def __init__(self, driver_id, work_hours, rest_hours=None):
        self.driver_id = driver_id
        self.work_hours = work_hours
        self.rest_hours = rest_hours if rest_hours else []

    def is_free(self, now):
        weekday_now = now.weekday()
        current_date = now.date()

        for day, start, finish in self.work_hours:
            if weekday_now == day:
                start_time = datetime.combine(current_date, datetime.strptime(start, "%H:%M").time())
                end_time = datetime.combine(current_date, datetime.strptime(finish, "%H:%M").time())

                if start_time <= now < end_time:
                    for r_day, r_start, r_end in self.rest_hours:
                        if weekday_now == r_day:
                            rest_start = datetime.combine(current_date, datetime.strptime(r_start, "%H:%M").time())
                            rest_end = datetime.combine(current_date, datetime.strptime(r_end, "%H:%M").time())

                            if rest_start <= now < rest_end:
                                return False
                    return True
        return False

In [12]:
class Vehicle:
    def __init__(self, vehicle_id, limit, driver):
        self.vehicle_id = vehicle_id
        self.limit = limit
        self.driver = driver
        self.current_passengers = 0
        self.running = True

    def check_driver(self, now):
        if self.driver is None or not self.driver.is_free(now):
            self.running = False

In [17]:
class RoutePlan:
    def __init__(self, start, end):
        self.start = start
        self.end = end
        self.current_time = start
        self.interval = timedelta(minutes=1)
        self.paths = []
        self.vehicles = []
        self.drivers = []
        self.total_riders = 0
        self.remaining_riders = 0
        self.log = []

    def add_station(self, station_name, time_to_next, stop_time, initial_waiting):
        self.paths.append({
            "station_name": station_name, "time_to_next": time_to_next,
            "stop_time": stop_time, "waiting_people": initial_waiting
        })

    def register_vehicle(self, vehicle, delay=0, start_position=0):
        vehicle.start_time = self.start + timedelta(minutes=delay)
        vehicle.start_pos = start_position
        self.vehicles.append(vehicle)

    def register_driver(self, driver):
        self.drivers.append(driver)

    def get_available_driver(self, now):
        for driver in self.drivers:
            if driver.is_free(now):
                return driver
        return None

    def is_rush_time(self):
        for start, end in [(7, 9), (18, 20)]:
            if start <= self.current_time.hour < end:
                return True
        return False

    def is_late_night(self):
        return self.current_time.hour < 6 or self.current_time.hour >= 23

    def run_simulation(self):
        print(f"Симуляция: {self.start.strftime('%Y-%d-%m %H:%M')}\n")

        vehicle_positions = {v.vehicle_id: v.start_pos for v in self.vehicles}
        vehicle_departure_times = {v.vehicle_id: v.start_time for v in self.vehicles}

        while self.current_time < self.end:
            if self.current_time.time() == time(0, 0):
                print(f"\n={self.current_time.strftime('%A, %Y-%d-%m')}")

            for station in self.paths:
                new_arrivals = 0
                if self.is_late_night() and random.random() < 0.1:
                    new_arrivals = random.randint(0, 1)
                elif self.is_rush_time():
                    new_arrivals = random.randint(3, 6)
                elif random.random() < 0.3:
                    new_arrivals = random.randint(1, 4)

                station["waiting_people"] += new_arrivals

            for vehicle in self.vehicles:
                if not vehicle.running:
                    next_driver = self.get_available_driver(self.current_time)
                    if next_driver:
                        vehicle.driver = next_driver
                        vehicle.running = True
                        print(f"Автобус {vehicle.vehicle_id} отправляется с водителем {next_driver.driver_id}.")
                    continue

                vehicle.check_driver(self.current_time)
                if not vehicle.running:
                    print(f"\nВремя: {self.current_time.strftime('%H:%M')}")
                    print(f"Автобус {vehicle.vehicle_id} остановился, водитель {vehicle.driver.driver_id} отработал смену.")
                    continue

                current_position = vehicle_positions[vehicle.vehicle_id]
                if self.current_time >= vehicle_departure_times[vehicle.vehicle_id]:
                    station = self.paths[current_position]
                    station_name = station["station_name"]
                    waiting_people = station["waiting_people"]

                    self.log.append({
                        "time": self.current_time.strftime('%H:%M'),
                        "day": self.current_time.strftime('%A'),
                        "station_name": station_name,
                        "vehicle_id": vehicle.vehicle_id
                    })

                    print(f"\nАвтобус {vehicle.vehicle_id} прибывает на остановку '{station_name}' <<{self.current_time.strftime('%H:%M')}>>")

                    available_space = vehicle.limit - vehicle.current_passengers
                    boarding_passengers = min(available_space, waiting_people)
                    vehicle.current_passengers += boarding_passengers
                    station["waiting_people"] -= boarding_passengers
                    self.total_riders += boarding_passengers
                    print(f"Вошло {boarding_passengers} пассажиров. Нагрузка: {vehicle.current_passengers}/{vehicle.limit} человек")

                    # Общее количество выходящих пассажиров
                    disembarking_passengers = random.randint(vehicle.current_passengers // 3, vehicle.current_passengers)
                    vehicle.current_passengers -= disembarking_passengers
                    print(f"Вышло {disembarking_passengers} пассажиров.")

                    # Обновление времени отправления
                    stop_duration = timedelta(minutes=station["stop_time"])
                    vehicle_departure_times[vehicle.vehicle_id] = self.current_time + stop_duration

                    # Обновление позиции и времени в пути
                    next_position = (current_position + 1) % len(self.paths)
                    vehicle_positions[vehicle.vehicle_id] = next_position
                    vehicle_departure_times[vehicle.vehicle_id] += timedelta(minutes=station["time_to_next"])

            self.current_time += self.interval

        for station in self.paths:
            self.remaining_riders += station["waiting_people"]

        print("\nКонец симуляции.")
        print(f"Всего перевезено пассажиров: {self.total_riders}")

    def display_schedule(self):
        print("\nРасписание")

        # Заголовок таблицы
        print(f"{'Остановка':<30} {'День':<15} {'Время':<10} {'Автобус':<15}")
        print("-" * 70)

        days_translation = {
            "Monday": "Понедельник",
            "Tuesday": "Вторник",
            "Wednesday": "Среда",
            "Thursday": "Четверг",
            "Friday": "Пятница",
            "Saturday": "Суббота",
            "Sunday": "Воскресенье"
        }

        schedule_by_station = {}
        for entry in self.log:
            station_name = entry["station_name"]
            if station_name not in schedule_by_station:
                schedule_by_station[station_name] = []
            schedule_by_station[station_name].append(
                (days_translation.get(entry["day"], entry["day"]), entry["time"], f"Автобус {entry['vehicle_id']}")
            )

        day_order = list(days_translation.values())

        for station_name, entries in schedule_by_station.items():
            for day, time, vehicle_info in sorted(entries, key=lambda x: (day_order.index(x[0]), x[1])):
                print(f"{station_name:<30} {day:<15} {time:<10} {vehicle_info:<15}")

def get_upcoming_monday(start_date=None):
    start_date = start_date or datetime.now()
    days_ahead = 0 if start_date.weekday() == 0 else (7 - start_date.weekday())
    return start_date + timedelta(days=days_ahead)

monday_start = get_upcoming_monday()
simulation_start = datetime.combine(monday_start, datetime.strptime("08:00", "%H:%M").time())
simulation_end = simulation_start + timedelta(days=7)

route_plan = RoutePlan(start=simulation_start, end=simulation_end)

route_plan.add_station("Станция Бирюлёво-Товарная", 3, 1, random.randint(20, 30))
route_plan.add_station("Медынская улица, 4", 2, 1, random.randint(20, 30))
route_plan.add_station("Медынская улица, 8", 2, 1, random.randint(20, 30))
route_plan.add_station("Медынская улица", 3, 1, random.randint(20, 30))
route_plan.add_station("Медынская улица, 12", 4, 1, random.randint(20, 30))
route_plan.add_station("Управа района Бирюлёво-Западное", 3, 1, random.randint(20, 30))
route_plan.add_station("Районный центр Бирюсинка", 3, 1, random.randint(20, 30))
route_plan.add_station("Востряковский проезд", 4, 1, random.randint(20, 30))
route_plan.add_station("Востряковский проезд, 15", 3, 1, random.randint(20, 30))
route_plan.add_station("Бирюлёво Западное", 5, 1, random.randint(20, 30))

drivers = [
    BusDriver(
        driver_id=i,
        work_hours=[(j, "08:35", "18:35") for j in range(7)] if i <= 5 else [(j, "05:35", "18:35") for j in range(1, 8, 2)],
        rest_hours=[(j, "13:00", "14:00") for j in range(7)] if i <= 5 else [(j, "12:00", "12:20") for j in range(1, 8, 2)] + [(j, "16:00", "16:20") for j in range(1, 8, 2)]
    )
    for i in range(1, 9)
]

for i in range(len(drivers)):
    route_plan.register_driver(drivers[i])

vehicle_limits = [90, 60, 90, 60, 60, 90, 60]
vehicles = [
    Vehicle(vehicle_id=9330 + i, limit=vehicle_limits[i], driver=drivers[i])
    for i in range(len(vehicle_limits))
]

for i, v in enumerate(vehicles):
    route_plan.register_vehicle(v, delay=i * 5, start_position=i)


# **Выполнение "простого" алгоритма**

In [18]:
route_plan.run_simulation()

Выходные данные были обрезаны до нескольких последних строк (5000).

Автобус 9331 прибывает на остановку 'Бирюлёво Западное' <<15:26>>
Вошло 0 пассажиров. Нагрузка: 0/60 человек
Вышло 0 пассажиров.

Автобус 9332 прибывает на остановку 'Бирюлёво Западное' <<15:26>>
Вошло 0 пассажиров. Нагрузка: 0/90 человек
Вышло 0 пассажиров.

Автобус 9333 прибывает на остановку 'Станция Бирюлёво-Товарная' <<15:27>>
Вошло 22 пассажиров. Нагрузка: 22/60 человек
Вышло 10 пассажиров.

Автобус 9334 прибывает на остановку 'Станция Бирюлёво-Товарная' <<15:27>>
Вошло 0 пассажиров. Нагрузка: 0/60 человек
Вышло 0 пассажиров.

Автобус 9335 прибывает на остановку 'Управа района Бирюлёво-Западное' <<15:29>>
Вошло 19 пассажиров. Нагрузка: 27/90 человек
Вышло 9 пассажиров.

Автобус 9336 прибывает на остановку 'Управа района Бирюлёво-Западное' <<15:29>>
Вошло 0 пассажиров. Нагрузка: 0/60 человек
Вышло 0 пассажиров.

Автобус 9333 прибывает на остановку 'Медынская улица, 4' <<15:31>>
Вошло 8 пассажиров. Нагрузка: 20/60

In [19]:
route_plan.display_schedule()

Выходные данные были обрезаны до нескольких последних строк (5000).
Медынская улица                Среда           12:16      Автобус 9335   
Медынская улица                Среда           12:16      Автобус 9336   
Медынская улица                Среда           12:31      Автобус 9333   
Медынская улица                Среда           12:31      Автобус 9334   
Медынская улица                Среда           12:35      Автобус 9331   
Медынская улица                Среда           12:35      Автобус 9332   
Медынская улица                Среда           12:39      Автобус 9330   
Медынская улица                Среда           12:58      Автобус 9335   
Медынская улица                Среда           12:58      Автобус 9336   
Медынская улица                Среда           14:11      Автобус 9333   
Медынская улица                Среда           14:11      Автобус 9334   
Медынская улица                Среда           14:17      Автобус 9331   
Медынская улица                Среда        

# **Логика генетического алгоритма**

In [20]:
# Типы водителей
workhours_1 = [(0, "08:35", "18:35"), (1, "08:35", "18:35"), (2, "08:35", "18:35"), (3, "08:35", "18:35"), (4, "08:35", "18:35")]
resthours_1 = [(0, "13:00", "14:00"), (1, "13:00", "14:00"), (2, "13:00", "14:00"), (3, "13:00", "14:00"), (4, "13:00", "14:00")]
workhours_2 = [(1, "05:35", "18:35"), (3, "05:35", "18:35"), (5, "05:35", "18:35")]
resthours_2 = [(1, "12:00", "12:20"), (1, "16:00", "16:20"), (3, "12:00", "12:20"), (3, "16:00", "16:20"),(5, "12:00", "12:20"), (5, "16:00", "16:20")]

In [4]:
class Genetic:
    def __init__(self, num_generations=50, population_size=20, mutation_rate=0.1):
        self.num_generations = num_generations
        self.population_size = population_size
        self.mutation_rate = mutation_rate
        self.population = []

    def initialize_population(self):
        self.population = []
        for _ in range(self.population_size):
            driver_types = [random.choice([1, 2]) for _ in range(8)]
            self.population.append(driver_types)

    def fitness(self, driver_types):
        schedule = self.create_new_schedule(driver_types)

        original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

        schedule.run_simulation()

        sys.stdout.close()
        sys.stdout = original_stdout

        return schedule.remaining_riders

    def select_parents(self, fitness_scores):
        total_fitness = sum(1 / (1 + score) for score in fitness_scores)
        probabilities = [(1 / (1 + score)) / total_fitness for score in fitness_scores]
        return random.choices(self.population, probabilities, k=2)

    def crossover(self, parent1, parent2):
        point = random.randint(1, len(parent1) - 1)
        child1 = parent1[:point] + parent2[point:]
        child2 = parent2[:point] + parent1[point:]
        return child1, child2

    def mutate(self, individual):
        if random.random() < self.mutation_rate:
            index = random.randint(0, len(individual) - 1)
            individual[index] = 1 if individual[index] == 2 else 2
        return individual

    def create_new_schedule(self, driver_types):
        start_date = get_upcoming_monday()
        start_time = datetime.combine(start_date, datetime.strptime("05:35", "%H:%M").time())
        end_time = start_time + timedelta(days=7)

        peak_hours = [(7, 9), (17, 20)]

        schedule = RoutePlan(start=start_time, end=end_time)

        stops = [("Станция Бирюлёво-Товарная", 3, 1, random.randint(0,15)),
                ("Медынская улица, 4", 2, 1, random.randint(0,15)),
                ("Медынская улица, 8", 2, 1, random.randint(0,15)),
                ("Медынская улица", 3, 1, random.randint(0,15)),
                ("Медынская улица, 12", 4, 1, random.randint(0,15)),
                ("Управа района Бирюлёво-Западное", 3, 1, random.randint(0,15)),
                ("Районный центр Бирюсинка", 3, 1, random.randint(0,15)),
                ("Востряковский проезд", 4, 1, random.randint(0,15)),
                ("Востряковский проезд, 15", 3, 1, random.randint(0,15)),
                ("Бирюлёво Западное", 5, 1, random.randint(0,15))]



        for stop in stops:
            schedule.add_station(*stop)

        drivers = []
        for i, driver_type in enumerate(driver_types):
            if driver_type == 1:
                drivers.append(BusDriver(driver_id=i + 1, work_hours=workhours_1, rest_hours=resthours_1))
            elif driver_type == 2:
                drivers.append(BusDriver(driver_id=i + 1, work_hours=workhours_1, rest_hours=resthours_2))

        for driver in drivers:
            schedule.register_driver(driver)

        for i, driver in enumerate(drivers):
            bus = Vehicle(vehicle_id=100 + i + 1, limit=40, driver=driver)
            schedule.register_vehicle(bus, delay=i * 5, start_position=i % len(schedule.paths))

        return schedule

    def run(self):
        self.initialize_population()

        for generation in range(self.num_generations):
            fitness_scores = [self.fitness(individual) for individual in self.population]

            new_population = []
            while len(new_population) < self.population_size:
                parent1, parent2 = self.select_parents(fitness_scores)
                child1, child2 = self.crossover(parent1, parent2)
                new_population.append(self.mutate(child1))
                if len(new_population) < self.population_size:
                    new_population.append(self.mutate(child2))

            self.population = new_population

            best_fitness = min(fitness_scores)
            best_solution = self.population[fitness_scores.index(best_fitness)]

        best_fitness = min(fitness_scores)
        best_solution = self.population[fitness_scores.index(best_fitness)]
        return best_solution



# **Выполнение генетического алгоритма**

In [9]:
genetic_algorithm = Genetic(num_generations=10, population_size=5, mutation_rate=0.1)
result = genetic_algorithm.run()

In [21]:
route_plan.display_schedule()

Выходные данные были обрезаны до нескольких последних строк (5000).
Медынская улица                Среда           12:16      Автобус 9335   
Медынская улица                Среда           12:16      Автобус 9336   
Медынская улица                Среда           12:31      Автобус 9333   
Медынская улица                Среда           12:31      Автобус 9334   
Медынская улица                Среда           12:35      Автобус 9331   
Медынская улица                Среда           12:35      Автобус 9332   
Медынская улица                Среда           12:39      Автобус 9330   
Медынская улица                Среда           12:58      Автобус 9335   
Медынская улица                Среда           12:58      Автобус 9336   
Медынская улица                Среда           14:11      Автобус 9333   
Медынская улица                Среда           14:11      Автобус 9334   
Медынская улица                Среда           14:17      Автобус 9331   
Медынская улица                Среда        